In [2]:
import numpy as np
from sklearn.datasets import load_boston

In [3]:
np.random.seed(1234)
RD_MEAN = 0
RD_STD = 0.0030
LEARNING_RATE = 0.001

In [4]:
def boston_SLP_exec(epoch_counts=1,mb_size=10,report=1):
    
    load_datasets()
    init_model()
    train_and_test(epoch_counts,mb_size,report)

In [5]:
def load_datasets():
    bs=load_boston()
    
    global data, input_size, output_size
    
    features=bs.data
    target = bs.target.reshape(-1,1)
    input_size, output_size=len(bs.feature_names), 1
    
    data=np.concatenate((features,target),axis =1)

In [6]:
def init_model():
    
    global weight, bias, input_size, output_size
    weight = np.random.normal(RD_MEAN,RD_STD,[input_size,output_size])
    bias = np.zeros(output_size)

In [7]:
def train_and_test(epoch_counts, mb_size, report):
    step_count=arange_dataset(mb_size) 
    test_X, test_y = get_test_data() 
    
    for epoch in range(epoch_counts):
      losses, accs = [],[]
      for n in range(step_count):
        train_X, train_y = get_train_data(mb_size,n)
        loss, acc=run_train(train_X, train_y)
        losses.append(loss)
        accs.append(acc)

      if report > 0 and (epoch +1) % report == 0:
        acc = run_test(test_X, test_y)
        print('Epoch {}: loss={:5.3f}, accuracy = {:5.3f}'.\
          format(epoch+1,np.mean(losses), np.mean(accs),acc))
    final_acc = run_test(test_X,test_y)
    print('\nFinal Test: final accuracy ={:5.3f}'.format(final_acc))

In [8]:
def arange_dataset(mb_size):
    global data, shuffle_map, begin_test_data
    shuffle_map=np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    
    step_counts = int(data.shape[0] *0.8) // mb_size
    begin_test_data = step_counts * mb_size

    return step_counts

In [9]:
def get_test_data():
    global data, shuffle_map, begin_test_data, output_size
    
    test_data = data[shuffle_map[begin_test_data:]]
    
    test_X = test_data[:,:-1]
    test_y = test_data[:,-1:]
    
    return test_X , test_y

In [14]:
def get_train_data(mb_size,nth):
  global data, shuffle_map, begin_test_data, output_size


  if nth ==0:
    np.random.shuffle(shuffle_map[:begin_test_data])
  train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]

  train_X = train_data[:,:-output_size]
  train_y = train_data[:,-output_size:]

  return train_X,train_y

In [15]:
def run_train(x, y):
  output, aux_nn = forward_neuralnet(x)
  loss, aux_pp = forward_postproc(output,y)
  accuracy = eval_accuracy(output, y)

  G_loss = 1.0 # 여기서는 이제 편미분 및 그레디언트를통하여 움직일 위치 만큼의 값을 찾는다.거꾸로 가야한다.

  G_output=backward_postproc(G_loss, aux_pp)
  backward_neuralnet(G_output, aux_nn)

  return loss, accuracy

def run_test(x,y):
  output, _ = forward_neuralnet(x)
  accuracy = eval_accuracy(output, y)
  return accuracy

In [16]:
def forward_neuralnet(x):
  global weight, bias
  output = np.matmul(x,weight) + bias
  return output, x

def backward_neuralnet(G_output, x):
  global weight, bias
  g_output_w = x.transpose()
  G_w = np.matmul(g_output_w, G_output)
  G_b = np.sum(G_output, axis = 0)

  weight -= LEARNING_RATE * G_w
  bias -= LEARNING_RATE * G_b

In [17]:
def forward_postproc(output,y):
  diff = output - y
  square = np.square(diff)
  loss = np.mean(square)
  return loss, diff

def backward_postproc(G_loss, diff):
  shape=diff.shape
  g_loss_square = np.ones(shape) / np.prod(shape)
  g_square_diff = 2 * diff
  g_diff_output = 1

  G_square=g_loss_square * G_loss 
  G_diff = g_square_diff* G_square 
  G_output = g_diff_output*G_diff
  return G_output

In [18]:
def eval_accuracy(output, y):
  mdiff=np.mean(np.abs((output - y)/y))
  print(y)
  print(output)
  print(mdiff)
  return 1 - mdiff

In [19]:
boston_SLP_exec()

[[13.1]
 [25. ]
 [18.5]
 [24.7]
 [23.7]
 [26.7]
 [21.7]
 [ 8.4]
 [10.4]
 [ 9.7]]
[[-0.58437384]
 [-0.59147088]
 [-0.05216615]
 [-1.05835759]
 [-0.59874468]
 [-0.54532122]
 [-0.73875209]
 [-3.77304781]
 [-3.60111525]
 [-2.80268703]]
1.1278037429369534
[[25.2]
 [12.7]
 [34.9]
 [21.7]
 [23. ]
 [23.1]
 [20.5]
 [50. ]
 [18.9]
 [16.1]]
[[ 8667.23895866]
 [15188.96121855]
 [ 9093.02782603]
 [15101.59159224]
 [ 9548.49580786]
 [ 9867.76127975]
 [10064.69834637]
 [11316.68411166]
 [ 9577.47677275]
 [ 9234.69285536]]
512.6340933725115
[[12.8]
 [34.6]
 [18.4]
 [20.6]
 [22.6]
 [31.6]
 [50. ]
 [24.8]
 [27.5]
 [23.4]]
[[-9229594.18658165]
 [-6197360.60084547]
 [-6048054.33293828]
 [-5782971.93932905]
 [-5716403.14816189]
 [-5890822.62512343]
 [-6722566.49933174]
 [-5689954.05996654]
 [-6040968.60023453]
 [-5803257.4216307 ]]
278052.754299779
[[22. ]
 [13.1]
 [27.9]
 [23.1]
 [20.6]
 [10.8]
 [12.1]
 [50. ]
 [22.5]
 [41.7]]
[[3.53224047e+09]
 [5.05100913e+09]
 [3.09137977e+09]
 [3.56005095e+09]
 [2.988